# Ćwiczenie 3

Celem ćwiczenia jest imlementacja metody [Minimax z obcinaniem alpha-beta](https://en.wikipedia.org/wiki/Alpha%E2%80%93beta_pruning) do gry Connect Four (czwórki).

W trakcie ćwiczenia można skorzystać z reposytorium z implementacją gry [Connect Four udostępnionym przez Jakuba Łyskawę](https://github.com/lychanl/two-player-games). Ewentualnie, można zaimplementować samemu grę Connect Four (ale, tak aby rozwiązanie miało ten sam interfejs co podany poniżej).

Implementację Minimax należy przetestować używając różną głębokość przeszukiwania. Implementacja Solvera musi zapewniać interfejs jak poniżej, ale można dodać dowolne metody prywatne oraz klasy wspomagające (jeżeli będą potrzebne).

Punktacja:
- Działająca metoda Minimax - **2 pkt**
- Działająca metoda Minimax z obcinaniem alpha-beta - **1.5 pkt**
- Analiza jakości solvera w zależności od głębokości przeszukiwania **1.5pkt**
    - można zaimplementować w tym celu wizualizację rozgrywki dwóch agentów, bądź kilka przykładów 'z ręki'
- Jakość kodu **2pkt**

Aby importowanie elementów z poniższej komórki działało należy umieścić tego notebooka w tym samym folderze co paczkę `two_player_games`:
```
├── LICENSE
├── README.md
├── minimax.ipynb # HERE
├── test
│   ├── __init__.py
│   ├── test_connect_four.py
│   ├── test_dots_and_boxes.py
│   └── test_pick.py
└── two_player_games
    ├── __init__.py
    ├── games
    │   ├── connect_four.py
    │   └── dots_and_boxes.py
    ├── move.py
    ├── player.py
    └── state.py
```

In [1]:
from typing import Tuple
import random

Wielkość planszy

In [2]:
ROW_COUNT = 6
COLUMN_COUNT = 7

In [3]:
class ConnectFour:
    empty = 0
    first_player = 1
    second_player = 2

    def __init__(self, columns, rows):
        self.rows = rows
        self.columns = columns
        self.current_player = self.first_player
        self.fields = [[self.empty for _ in range(rows)] for _ in range(columns)]
        self._history = []
        self.segments = self._make_segments()

    def get_moves(self):
        return [i for i, column in enumerate(self.fields) if not column[-1]]

    def make_move(self, column):
        for r in range(self.rows):
            if not self.fields[column][r]:
                self.fields[column][r] = self.current_player
                self._history.append((column, r))
                self._swap_player()
                return
        raise ValueError(f"unavailable move {column}")

    def undo_move(self):
        c, r = self._history.pop()
        self.fields[c][r] = 0
        self._swap_player()

    def _make_segments(self):
        segments = []
        # vertical
        for c in range(self.columns):
            for s in range(self.rows - 3):
                segments.append(tuple((c, s + i) for i in range(4)))
        # horizontal
        for r in range(self.rows):
            for s in range(self.columns - 3):
                segments.append(tuple((s + i, r) for i in range(4)))
        # diagonal right-up
        for c in range(self.columns - 3):
            for r in range(self.rows - 3):
                segments.append(tuple((c + i, r + i) for i in range(4)))
        # diagonal right-down
        for c in range(self.columns - 3):
            for r in range(3, self.rows):
                segments.append(tuple((c + i, r - i) for i in range(4)))
        return segments

    def at(self, coords):
        return self.fields[coords[0]][coords[1]]

    def get_winner(self):
        for segment in self.segments:
            for player in (self.first_player, self.second_player):
                if all(self.at(segment[i]) == player for i in range(4)):
                    return player
        return None

    def draw(self):
        return len(self._history) == self.rows * self.columns and not self.get_winner()

    def __str__(self):
        def field_to_string(f):
            if not f:
                return " "
            elif f == self.first_player:
                return "a"
            else:
                return "b"

        def coord_to_string(c, r):
            s = field_to_string(self.fields[c][r])
            return s.upper() if self._history and self._history[-1] == (c, r) else s

        # taken from https://github.com/lychanl/two-player-games
        transposed = zip(*self.fields)
        text = "\n".join(
            reversed(
                [
                    "".join(f"[{coord_to_string(c, r)}]" for c, _ in enumerate(row))
                    for r, row in enumerate(transposed)
                ]
            )
        )

        if self.get_winner():
            return f"Player {field_to_string(self.get_winner()).upper()} won\n{text}"
        elif self.draw():
            return f"Draw\n{text}"
        else:
            return f"Current player: {field_to_string(self.current_player)}\n{text}"

    def is_finished(self):
        return self.get_winner() or all(self.fields[c][-1] for c in range(self.columns))

    def _swap_player(self):
        self.current_player = (
            self.first_player
            if self.current_player == self.second_player
            else self.second_player
        )

In [4]:
class MinMaxSolver:
    def __init__(self, game: ConnectFour, depth, verbose=False):
        self.game = game
        self.depth = depth
        self.verbose = verbose

    def evaluate_position(self) -> float:
        score = 0
        for segment in self.game.segments:
            counts = {
                self.game.empty: 0,
                self.game.first_player: 0,
                self.game.second_player: 0,
            }
            for i in range(4):
                counts[self.game.at(segment[i])] += 1
            weights = [1, 5, float("inf")]
            start = 4 + 1 - len(weights)
            for player in (self.game.first_player, self.game.second_player):
                for i, w in enumerate(weights, start=start):
                    if counts[player] == i and counts[self.game.empty] == 4 - i:
                        score += self._sign(player) * w
        return score

    def minimax(self, depth, alpha: float, beta: float) -> Tuple[int | None, float]:
        """Returns column index and score"""
        possible_winner = self.game.get_winner()
        if possible_winner:
            return None, float("inf") * self._sign(possible_winner)
        if depth == 0:
            return None, self.evaluate_position()
        if self.game.current_player == ConnectFour.first_player:
            value = float("-inf")
            best_move = None
            for move in self.game.get_moves():
                self.game.make_move(move)
                new_value = self._minimax_value(depth - 1, alpha, beta)
                self.game.undo_move()
                if new_value > value:
                    value = new_value
                    best_move = move
                alpha = max(alpha, value)
                if value >= beta:
                    break
            return best_move, value
        else:
            value = float("+inf")
            best_move = None
            for move in self.game.get_moves():
                self.game.make_move(move)
                new_value = self._minimax_value(depth - 1, alpha, beta)
                self.game.undo_move()
                if new_value < value:
                    value = new_value
                    best_move = move
                beta = min(beta, value)
                if value <= alpha:
                    break
            return best_move, value

    def _minimax_value(self, *args, **kwargs) -> float:
        _, value = self.minimax(*args, **kwargs)
        return value

    def _sign(self, player=None):
        if player is None:
            player = self.game.current_player
        return {self.game.first_player: 1, self.game.second_player: -1}[player]

    def get_best_move(self) -> int:
        move, value = self.minimax(self.depth, float("-inf"), float("+inf"))
        if self.verbose:
            print(f"Current value: {value}")
        if move is None:
            move = random.choice(self.game.get_moves())
        return move

Rozgrywka

In [5]:
random.seed(0)

# Analiza jakości

In [6]:
import itertools


def test_skill_diff(
    player=ConnectFour.second_player, min_skill=2, max_skill=5, max_skill_diff=4
):
    won, drew, lost = [], [], []
    is_first = ConnectFour.first_player == player
    other_player = ConnectFour.second_player if is_first else ConnectFour.first_player
    for skill_diff in range(1, max_skill_diff + 1):
        for depth in range(min_skill, max_skill - skill_diff + 1):
            game = ConnectFour(COLUMN_COUNT, ROW_COUNT)
            mms = MinMaxSolver(game, depth)
            skills = (
                (depth + skill_diff, depth) if is_first else (depth, depth + skill_diff)
            )
            print(f"A lvl {skills[0]}, B lvl {skills[1]}")
            depth_iter = itertools.cycle(skills)
            while not game.is_finished():
                mms.depth = next(depth_iter)
                game.make_move(mms.get_best_move())
            print(game)
            print()
            winner = game.get_winner()
            if winner == player:
                won.append(skills)
            elif winner == other_player:
                lost.append(skills)
            else:
                drew.append(skills)

    def print_stat(matches, caption):
        if matches:
            print(f"{caption}:")
            for skill_a, skill_b in matches:
                print(f"\tA lvl {skill_a} vs B lvl {skill_b}")

    print_stat(lost, "Lost")
    print_stat(drew, "Drew")

In [7]:
test_skill_diff(ConnectFour.second_player)

A lvl 2, B lvl 3


Player A won
[a][b][b][ ][b][ ][b]
[a][a][a][A][a][ ][a]
[b][b][a][a][b][ ][b]
[b][a][a][b][a][ ][a]
[a][b][b][a][b][b][b]
[a][a][b][b][a][b][a]

A lvl 3, B lvl 4
Player B won
[a][a][b][a][b][a][B]
[a][b][b][b][a][b][a]
[b][b][a][a][b][b][b]
[b][a][b][b][a][a][a]
[a][b][a][a][b][a][b]
[a][a][b][a][a][b][b]

A lvl 4, B lvl 5
Player B won
[a][ ][a][b][ ][ ][a]
[a][ ][b][b][ ][a][b]
[b][ ][a][a][ ][b][a]
[a][B][b][b][ ][a][b]
[b][a][b][a][b][b][b]
[a][a][a][b][a][b][a]

A lvl 2, B lvl 4
Draw
[a][a][b][a][b][b][B]
[a][b][b][b][a][a][a]
[b][b][a][a][b][b][b]
[b][a][b][b][a][a][a]
[a][b][a][a][b][b][b]
[a][a][b][a][a][a][b]

A lvl 3, B lvl 5
Player B won
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][a][a][B][ ]
[b][ ][a][a][a][b][ ]
[a][ ][a][b][b][b][ ]
[a][b][a][b][b][b][a]

A lvl 2, B lvl 5
Player B won
[b][ ][a][a][ ][ ][ ]
[a][ ][b][b][ ][ ][ ]
[b][ ][b][b][ ][ ][ ]
[a][B][b][a][ ][ ][ ]
[b][b][a][b][ ][a][a]
[a][a][a][b][a][b][a]

Lost:
	A lvl 2 vs B lvl 3
Drew:
	A lvl 2 vs B l

In [8]:
test_skill_diff(ConnectFour.first_player)

A lvl 3, B lvl 2
Player A won
[ ][ ][ ][ ][ ][ ][ ]
[b][a][ ][ ][ ][ ][ ]
[a][b][b][A][a][ ][ ]
[a][b][a][a][b][ ][ ]
[b][a][b][b][a][ ][ ]
[a][a][b][a][b][b][ ]

A lvl 4, B lvl 3
Player A won
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][b][ ][ ][ ][ ]
[ ][b][a][b][ ][ ][ ]
[A][a][a][a][ ][ ][ ]
[a][b][b][a][ ][ ][ ]
[b][b][a][a][b][ ][ ]

A lvl 5, B lvl 4
Player A won
[b][ ][b][a][ ][a][ ]
[a][ ][b][b][ ][a][ ]
[b][ ][b][b][a][a][ ]
[a][A][a][a][b][b][ ]
[b][b][b][a][a][a][ ]
[b][b][a][a][b][a][ ]

A lvl 4, B lvl 2
Player A won
[b][b][a][ ][ ][a][a]
[a][a][b][A][ ][b][b]
[b][b][a][b][ ][b][a]
[b][a][a][a][ ][a][b]
[a][a][b][b][ ][a][a]
[b][b][a][a][ ][b][b]

A lvl 5, B lvl 3
Player B won
[b][b][a][b][b][B][b]
[a][a][a][b][b][a][a]
[b][b][b][a][b][b][b]
[a][a][a][b][a][a][a]
[a][b][a][a][a][b][b]
[b][b][a][a][a][b][a]

A lvl 5, B lvl 2
Player B won
[ ][b][a][b][a][a][ ]
[ ][b][a][b][b][b][ ]
[ ][a][b][b][a][a][b]
[ ][b][b][a][b][a][a]
[B][a][a][a][b][b][b]
[a][b][a][a][b][a][a]

Lost:
	A lvl 5 vs B lv

Gracz przeszukujący głębiej zwykle wygra, niezależnie od tego czy zaczął drugi czy pierwszy. Czasami zdarzy się remis lub przegrana nawet przy znaczącej różnicy głębokości (gracz zaczynający o głębokości 5 przegrał z graczem o głębokości 2). Wynika to z tego, że głębokość 5 nie jest wystarczająco duża do dobrej oceny planszy. Algorytm polega zbyt bardzo na heurystyce, która nie gwarantuje wygranej.

# Niefortunna gra

In [9]:
game = ConnectFour(COLUMN_COUNT, ROW_COUNT)
mms = MinMaxSolver(game, 6, verbose=True)
depth_iter = itertools.cycle((5, 2))
while not game.is_finished():
    mms.depth = next(depth_iter)
    game.make_move(mms.get_best_move())
    print(game)

Current value: 2
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][A][ ][ ][ ][ ]
Current value: 1
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][B][a][ ][ ][ ][ ]
Current value: 4
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][A][ ][ ][ ][ ]
[ ][b][a][ ][ ][ ][ ]
Current value: 2
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][B][ ][ ][ ][ ]
[ ][ ][a][ ][ ][ ][ ]
[ ][b][a][ ][ ][ ][ ]
Current value: 4
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][b][ ][ ][ ][ ]
[ ][A][a][ ][ ][ ][ ]
[ ][b][a][ ][ ][ ][ ]
Current value: 1
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][B][b][ ][ ][ ][ ]
[ ][a][a][ ][ ][ ][ ]
[ ][b][a][ ][ ][ ][ 

Możemy zauważyć, że pod koniec zostają 2 kolumny, które są prawie puste. Ich przeszukanie wymagałoby głębokości ~12.

# AI vs AI

In [10]:
game = ConnectFour(COLUMN_COUNT, ROW_COUNT)
mms = MinMaxSolver(game, 6, verbose=True)

while not game.is_finished():
    game.make_move(mms.get_best_move())
    print(game)

Current value: 0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][A][ ][ ][ ]
Current value: 4
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][B][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Current value: -1
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][A][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Current value: 5
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][B][a][ ][ ][ ]
Current value: 0
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][A][b][ ][ ][ ]
[ ][ ][b][a][ ][ ][ ]
Current value: 6
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][B][a][ ][ ][ ]
[ ][ ][a][b][ ][ ][ ]
[ ][ ][b][a][ ][ ][

# random vs AI

In [11]:
game = ConnectFour(COLUMN_COUNT, ROW_COUNT)
mms = MinMaxSolver(game, 6)

while not game.is_finished():
    game.make_move(random.choice(game.get_moves()))
    print(game)
    game.make_move(mms.get_best_move())
    print(game)

Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][A]
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][B][ ][ ][a]
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][A][a]
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][B][ ][b][ ][a][a]
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][b][A][b][ ][a][a]
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][B][ ][ ][ ][ ][ ]
[ ][b][a][b][ ][a][a]
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][

# player vs AI

In [12]:
import time

game = ConnectFour(COLUMN_COUNT, ROW_COUNT)
mms = MinMaxSolver(game, 6)

if input("Play with AI? [y/N]").lower().startswith("y"):
    while not game.is_finished():
        print(game)
        for i in range(COLUMN_COUNT):
            print(f" {i} ", end="")
        print()
        time.sleep(1)
        i = int(input("Move? -1 to undo "))
        if i < 0:
            game.undo_move()
            game.undo_move()
            continue
        game.make_move(i)
        print(game)
        game.make_move(mms.get_best_move())
        print(game)
        if game.is_finished() and game.get_winner() == game.second_player:
            print("You lost, automatically undoing")
            game.undo_move()
            game.undo_move()